In [1]:
# 📦 Import required packages
import yfinance as yf
import pandas as pd
import json 
from datetime import datetime




In [2]:
# 📋 Load the most recent strategy from trade_log.json
with open("data/trade_log.json", "r") as f:
    trade_log = json.load(f)
    strategy = trade_log[-1]  # Get the last logged trade
    print("Loaded strategy:", strategy["strategy_name"])



Loaded strategy: Long Straddle


In [3]:
# 📥 Download historical data using yfinance
ticker = strategy["ticker"]
entry_date = pd.to_datetime(strategy["timestamp"]).normalize()

# Download 30 days of history before and after the trade timestamp
start_date = (entry_date - pd.Timedelta(days=30)).strftime('%Y-%m-%d')
end_date = (entry_date + pd.Timedelta(days=30)).strftime('%Y-%m-%d')

df = yf.download(ticker, start=start_date, end=end_date)
df.reset_index(inplace=True)
df["date"] = pd.to_datetime(df["Date"]).dt.date



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
# 🔁 Backtest the strategy against historical highs/lows
def backtest_strategy(df, strategy):
    entry_price = strategy["entry_price"]
    target_price = entry_price * (1 + strategy["target_pct"])
    stop_price = entry_price * (1 - strategy["stop_pct"])

    for row in df.itertuples():
        high = getattr(row, "high", None)
        low = getattr(row, "low", None)
        date = getattr(row, "date", None)

        if high is None or low is None or date is None:
            continue  # Skip rows with missing values

        if high >= target_price:
            return {
                "outcome": "Target Hit",
                "exit_price": target_price,
                "exit_date": date.strftime("%Y-%m-%d")
            }
        elif low <= stop_price:
            return {
                "outcome": "Stop Hit",
                "exit_price": stop_price,
                "exit_date": date.strftime("%Y-%m-%d")
            }

    return {
        "outcome": "Open",
        "exit_price": entry_price,
        "exit_date": None
    }




In [5]:
# 🧪 Run backtest and attach result to strategy
result = backtest_strategy(df, strategy)
strategy["backtest_result"] = result

print("📊 Backtest Result:", result)



📊 Backtest Result: {'outcome': 'Open', 'exit_price': 3.25, 'exit_date': None}
